In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [10]:
# File path
file_path = "/data/WA_Fn-UseC_-Telco-Customer-Churn.csv"

In [3]:
# 1. Initialize Spark Session
spark = SparkSession.builder \
    .appName("TelcoCustomerChurnPrediction") \
    .master("local[*]") \
    .getOrCreate()


25/04/25 23:14:34 WARN Utils: Your hostname, mbp.local resolves to a loopback address: 127.0.0.1; using 10.20.5.195 instead (on interface en0)
25/04/25 23:14:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/25 23:14:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# 2. Load the Telco Customer Churn dataset
def load_data(file_path):
    try:
        df = spark.read.csv(file_path, header=True, inferSchema=True)
        print("Dataset loaded successfully.")
        return df
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

In [5]:
# 3. Data Preprocessing
def preprocess_data(df):
    # Drop customerID as it's not needed for prediction
    df = df.drop("customerID")
    
    # Handle missing values (TotalCharges may have empty strings or nulls)
    df = df.filter(col("TotalCharges") != " ")  # Remove rows with empty TotalCharges
    df = df.withColumn("TotalCharges", col("TotalCharges").cast("float"))
    
    # Define categorical and numerical columns
    categorical_columns = [
        "gender", "SeniorCitizen", "Partner", "Dependents", "PhoneService",
        "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup",
        "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies",
        "Contract", "PaperlessBilling", "PaymentMethod"
    ]
    numerical_columns = ["tenure", "MonthlyCharges", "TotalCharges"]
    
    # Index categorical columns
    indexed_columns = [f"{col}_indexed" for col in categorical_columns]
    indexers = [
        StringIndexer(inputCol=col, outputCol=f"{col}_indexed", handleInvalid="keep")
        for col in categorical_columns
    ]
    
    for indexer in indexers:
        df = indexer.fit(df).transform(df)
    
    # Index target variable (Churn)
    df = StringIndexer(inputCol="Churn", outputCol="label").fit(df).transform(df)
    
    # Assemble features
    feature_columns = numerical_columns + indexed_columns
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="raw_features", handleInvalid="skip")
    df = assembler.transform(df)
    
    # Scale features
    scaler = StandardScaler(inputCol="raw_features", outputCol="features", withStd=True, withMean=True)
    df = scaler.fit(df).transform(df)
    
    return df, feature_columns

In [16]:
# 4. Main function
def main():
    # Path to the dataset (update this to your local path)
    file_path = "WA_Fn-UseC_-Telco-Customer-Churn.csv"
    
    # Load data
    df = load_data(file_path)
    if df is None:
        print("Exiting due to data loading error.")
        return
    # Preprocess data
    df_processed, feature_columns = preprocess_data(df)
    
    # Train-test split
    train_df, test_df = df_processed.randomSplit([0.8, 0.2], seed=42)
    
    # Train Random Forest model
    rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=100, seed=42)
    model = rf.fit(train_df)
    
    # Make predictions
    predictions = model.transform(test_df)
    
    # Evaluate model
    evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
    evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
    evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
    evaluator_roc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
    
    accuracy = evaluator_accuracy.evaluate(predictions)
    precision = evaluator_precision.evaluate(predictions)
    recall = evaluator_recall.evaluate(predictions)
    roc_auc = evaluator_roc.evaluate(predictions)
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': model.featureImportances.toArray()
    }).sort_values('importance', ascending=False)
    
    # Plot feature importance
    plt.figure(figsize=(12, 8))
    sns.barplot(x='importance', y='feature', data=feature_importance)
    plt.title('Feature Importance in Telco Churn Prediction')
    plt.tight_layout()
    plt.savefig('telco_feature_importance.png')
    plt.close()
    
    # Save feature importance data
    feature_importance.to_csv('telco_feature_importance.csv', index=False)
    print("\nFeature importance plot saved as 'telco_feature_importance.png'")
    print("Feature importance data saved as 'telco_feature_importance.csv'")
    
    # Stop Spark session
    spark.stop()

In [17]:
if __name__ == "__main__":
    main()

Error loading dataset: [PATH_NOT_FOUND] Path does not exist: file:/Users/zaw/github_repos/Spark/WA_Fn-UseC_-Telco-Customer-Churn.csv.
Exiting due to data loading error.
